# Suplari Data Exercise

This notebook answers the following four questions: 

1. What are the top 10 suppliers for 2017 and how much is spent on each supplier?

2. Which business unit was the top spender so far in 2017?

3. Compare the top 10 suppliers in 2017 to the top suppliers in 2016

4. How many transactions were recorded for 2015, 2016, and 2017 respectively?

To do so, I used the datasets located in the `\Data` folder.

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

### Load datasets

In [2]:
mapping_bu = pd.read_csv('Data/techtrades_bu.csv')
mapping_account = pd.read_csv('Data/techtrades_accounts.csv')
data2015 = pd.read_csv('Data/2015pcard_data.csv')
data2016 = pd.read_csv('Data/2016pcard_data.csv')
data2017 = pd.read_csv('Data/2017pcard_data.csv')

## 1. What are the top 10 suppliers for 2017 and how much is spent on each supplier?

### Exploring 2017 dataset columns

In [3]:
data2017.head(4)

,transactionid,amount,transactiondate,vendor,merchantcode,bu,account,ExpenseType,costcenter
0,17pcp8865,18.929075,7/10/2017,76,15100,100,8740,Building Suppliles,Corporate IT
1,17pcp17762,53.014500,6/11/2017,76,15100,100,8740,Building Suppliles,Corporate IT
2,17pcp154402,70.662625,7/29/2017,76,15100,100,8416,Car Rental,Corporate IT
3,17pcp155646,24.557775,5/27/2017,76,15100,100,8416,Car Rental,Corporate IT


In [4]:
data2017.vendor.unique()

array(['76', '01 BARTELL DRUGS', "025 IVAR'S SEATAC", ...,
       "LITTLE ANTHONY'S", "LONGO'S", 'UPEXPRESS.COM'], dtype=object)

In [5]:
with np.printoptions(threshold=np.inf):
    print(data2017.vendor.unique())

['76' '01 BARTELL DRUGS' "025 IVAR'S SEATAC" '1057 JESSIE'
 '32750 - BWI DAILY GARAGE' '3380 SLS LAS VEGAS' '443 N WABASH'
 '4TE*CULLIGAN OF ONTARIO' '51449 - 1616 SANSOM ST'
 '652 FLIGHT TERMINAL A' '7 ELEVEN STORE' '72711 - THE MAXWELL' '7-ELEVEN'
 '7TH AND PIKE GARAGE' '81411 - THE GROVE - SELF'
 '81577 - THE AMERICANA AT' '81787 - CALIFORNIA MART -'
 '82072 - 1600 SEVENTH AVE' '82636 - 100 N. LA CIENEGA'
 '85C BAKERY CAFE USA' '91 EXPRESS LANES REBILLS' '99 CENTS ONLY STORES'
 '99228 - 149 NEWBURY STREE' '99-CENTS-ONLY' 'A&W'
 'A-1 AIRLINE TAXI AND VAN' 'A4U' 'AA INFLIGHT VISA FACET'
 'AAROPORT LIMOUSINE SER' 'ABC COCINA' 'ABC DISPOSAL SYSTEMS INC'
 'ABC HAWAII' 'ABC KITCHEN' 'ABM ONSITE THE BLOC'
 'ABM ONSITE WESTFIELD MALL' 'ABM PARKING LIBERTY CENTE'
 'ABM PARKING SAN JOSE AIRP' 'ABM PARKING SHOPS AT MIDT'
 'ABSOLUT DIESEL REPAIR LL' 'ABSOPURE WATER COMPANY' 'AC MOORE STR'
 'ACCENTS' 'ACCESS' 'ACCESS INFO MGMT' 'ACE HOTEL DOWNTOWN LA'
 'ACE HOTEL DOWNTOWN LA F&B' 'ACE HOTEL NEW 

We have seven columns:
- `transactionid`: the id of the transaction.
- `amount`: the cost of the transaction.
- `transactiondate`: the date of the transaction.
- `vendor`: name of the vendor
- `merchantcode`: a code specific to each vendor. 
- `bu`: bussiness unit of the vendor that made the transaction.
- `account`: name of the vendor account (e.g., Professional Services, Advertisement).
- `ExpenseType`: the nature of the transaction.
- `costcenter`: cost center (a department within a company) of the vendor. 

In [6]:
data2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189858 entries, 0 to 189857
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   transactionid    189858 non-null  object 
 1   amount           189858 non-null  float64
 2   transactiondate  189858 non-null  object 
 3   vendor           189858 non-null  object 
 4   merchantcode     189858 non-null  int64  
 5   bu               189858 non-null  int64  
 6   account          189858 non-null  int64  
 7   ExpenseType      189858 non-null  object 
 8   costcenter       189858 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 13.0+ MB


- `transactiondate` is saved as an object. If needed, we could transform it later to a datetime object.
- Also, `merchantcode`, `bu`, and `account` are stored as an `int64` but they should be objects. We have mappings for the last two, so let's use their text names.
- Finally, `costcenter` also has a mapping, but it's almost identical to the values we currently have. 


In [7]:
# Change type of merchant code
data2017.merchantcode = data2017.merchantcode.astype('object')

In [8]:
mapping_bu.head()

,buid,name
0,100,"Techtrades, Inc. (HQ)"
1,110,Techtrades LTD (UK)
2,120,Techtrades NK (Japan)
3,200,"Zeus Banking, Inc."


In [9]:
#Mapping bu
mapping_bu_dict = mapping_bu.set_index('buid')['name'].to_dict()
data2017.bu = data2017.bu.map(mapping_bu_dict, na_action='ignore')

In [10]:
len(mapping_account)

41

In [11]:
mapping_account.head()

,accountcode,accountname
0,1250,Employee Receivables
1,1500,Other Current Assets
2,1540,Prepaid Vendor Amounts
3,1590,Other Prepaid Expenses
4,6150,Freight Cost


In [12]:
mapping_account.tail()

,accountcode,accountname
36,8645,IT Maintenance
37,8650,Equipment Mantenance and Repair
38,8720,Building Maintenance
39,8730,Property Taxes
40,8740,Building Suppliles


In [13]:
#Mapping account
mapping_account_dict = mapping_account.set_index('accountcode')['accountname'].to_dict()
data2017.account = data2017.account.map(mapping_account_dict, na_action='ignore')

In [14]:
# Check dataset again
data2017.head(4)

,transactionid,amount,transactiondate,vendor,merchantcode,bu,account,ExpenseType,costcenter
0,17pcp8865,18.929075,7/10/2017,76,15100,"Techtrades, Inc. (HQ)",Building Suppliles,Building Suppliles,Corporate IT
1,17pcp17762,53.014500,6/11/2017,76,15100,"Techtrades, Inc. (HQ)",Building Suppliles,Building Suppliles,Corporate IT
2,17pcp154402,70.662625,7/29/2017,76,15100,"Techtrades, Inc. (HQ)",Car Rental,Car Rental,Corporate IT
3,17pcp155646,24.557775,5/27/2017,76,15100,"Techtrades, Inc. (HQ)",Car Rental,Car Rental,Corporate IT


In [15]:
data2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189858 entries, 0 to 189857
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   transactionid    189858 non-null  object 
 1   amount           189858 non-null  float64
 2   transactiondate  189858 non-null  object 
 3   vendor           189858 non-null  object 
 4   merchantcode     189858 non-null  object 
 5   bu               189858 non-null  object 
 6   account          189858 non-null  object 
 7   ExpenseType      189858 non-null  object 
 8   costcenter       189858 non-null  object 
dtypes: float64(1), object(8)
memory usage: 13.0+ MB


### Exploring null values

In [16]:
data2017.isnull().sum()

transactionid      0
amount             0
transactiondate    0
vendor             0
merchantcode       0
bu                 0
account            0
ExpenseType        0
costcenter         0
dtype: int64

No missing values! Great!

### Checking for duplicate ids

In [17]:
data2017.transactionid.duplicated().sum() # The result of the duplicated() is a boolean Series, and we can add them up to count the number of duplicates. Behind the theme, True get converted to 1 and False get converted to 0, then it adds them up.

0

No duplicate ids. Cool.

### Checking the amount column

In [18]:
data2017.amount.max()

765407.166

In [19]:
data2017.amount.min()

-20147.98775

In [ ]:
sns.displot(data=data2017, x='amount')

In [ ]:
sns.displot(data=data2017, x='amount', bins=20)

## 2. Which business unit was the top spender so far in 2017?

## 3. Compare the top 10 suppliers in 2017 to the top suppliers in 2016

## 4. How many transactions were recorded for 2015, 2016, and 2017 respectively?